In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import json
import math
import csv
import plotly.graph_objects as go

In [2]:
airdf = pd.read_csv('CSV/runways.csv')
airdf.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,country_name,region_name,municipality,...,midx,midy,le_ident,le_latitude_deg,le_longitude_deg,le_heading_degT,he_ident,he_latitude_deg,he_longitude_deg,he_heading_degT
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,United States,California,Los Angeles,...,33.950600,-118.416500,6,33.949100,-118.431000,83.0,24,33.952099,-118.402000,263.0
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,United States,Illinois,Chicago,...,41.989601,-87.905151,4,41.981701,-87.913902,39.0,22,41.997501,-87.896400,219.0
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,United States,New York,New York,...,40.635401,-73.775150,4,40.622002,-73.785599,31.0,22,40.648800,-73.764702,211.0
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,United States,Georgia,Atlanta,...,33.649502,-84.424252,8,33.649502,-84.439003,90.0,26,33.649502,-84.409500,270.0
4,3878,KSFO,large_airport,San Francisco International Airport,37.618999,-122.375000,13.0,United States,California,San Francisco,...,37.618101,-122.376003,1,37.609001,-122.382004,28.0,19,37.627201,-122.370003,208.0


In [3]:
airdict = airdf.set_index('id').T.to_dict('list')
print(airdict[3632])

['KLAX', 'large_airport', 'Los Angeles International Airport', 33.942501, -118.407997, 125.0, 'United States', 'California', 'Los Angeles', 240920, 8925.0, 150.0, 'CON', 1, 0.1033938, 46.1941368, 33.95059967, -118.4165001, 6, 33.94910049, -118.4309998, 83.0, 24, 33.95209885, -118.4020004, 263.0]


In [4]:
headdict = {}

for i in airdict:
    headdict[airdict[i][18]] = []
for i in airdict:
    headdict[airdict[i][18]].append(
        [
            airdict[i][0],
            airdict[i][14],
            airdict[i][15],
            airdict[i][16],
            airdict[i][17],
            airdict[i][18],
            airdict[i][19],
            airdict[i][20],
            airdict[i][23],
            airdict[i][24],
        ]
    )


In [5]:
for i in headdict:
    print(f"{i}: {len(headdict[i])}")

6: 647
4: 752
8: 728
1: 630
13: 489
17: 495
7: 622
5: 772
3: 668
14: 441
10: 516
9: 907
2: 530
11: 433
12: 481
15: 430
16: 412
18: 570


In [6]:
segs = []
colors = []
my_cmap = plt.get_cmap('jet')

for i in headdict:
    if len(headdict[i]) > 1:
        for j in range(len(headdict[i])):
            x1 = headdict[i][j][7]
            y1 = headdict[i][j][6]
            x2 = headdict[i][j][9]
            y2 = headdict[i][j][8]
            c = headdict[i][j][5]

            colors.append(my_cmap(c))
            segs.append(((x1, y1), (x2, y2)))

fig, ax = plt.subplots()
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)

ln_coll = LineCollection(segs, colors=colors, linestyle='solid')

ax.add_collection(ln_coll)
fig.set_size_inches(20, 10)
fig.set_dpi(700)

plt.show()

In [7]:
with open('JSON/headings.json', 'w') as file:
    file.write(json.dumps(headdict))

In [8]:
with open("JSON/headings.json", "r") as h:
    headdict = json.load(h)

In [9]:
RADIUS = 0.001
inter = []
touch = []
outside = []
entrylst = []
enddict = {}

In [10]:
def checkCollision(a, b, c, x, y, radius, entr1, entr2):

    dist = (abs(a * x + b * y + c)) / math.sqrt(a * a + b * b)

    if radius == dist:
        touch.append(1)
        entrylst.append([entr1, entr2])
    elif radius > dist:
        inter.append(1)
        if entr1[0] != entr2[0]:
            entrylst.append([entr1, entr2])
    else:
        outside.append(1)

In [11]:
for heading in range(1, len(headdict) + 1):
    cur_heading = headdict[str(heading)]
    for i in range(0, len(cur_heading)):
        x = cur_heading[i][4]
        y = cur_heading[i][3]
        entr1 = cur_heading[i]

        for k in range(0, len(cur_heading)):
            a = cur_heading[k][1]
            b = -1
            c = cur_heading[k][2]
            entr2 = cur_heading[k]

            checkCollision(a, b, c, x, y, RADIUS, entr1, entr2)
            
print(f"touch:{len(touch)}\ninter:{len(inter)}\noutside:{len(outside)}")
print(len(entrylst))


touch:0
inter:10741
outside:6476174
218


In [12]:
for i in entrylst:
    enddict[i[0][5]] = []

for i in entrylst:
    enddict[i[0][5]].append(i)

In [13]:
print(enddict[1][1])

[['K1M5', 4.0960648, 390.8080504, 36.59285164, -86.4769516, 1, 36.58610153, -86.47859955, 36.59960175, -86.47530365], ['KGZH', 9.1003185, 823.5692775, 31.4147501, -87.0469017, 1, 31.40929985, -87.04750061, 31.42020035, -87.0463028]]


In [14]:
with open("JSON/intersecting.json", "w") as l:
    json.dump(enddict, l)

In [15]:
with open('JSON/intersecting.json','r') as f:
    headingdict = json.load(f)

In [16]:
print(headingdict["1"][0][0][0],headingdict["1"][0][0][4])
print(headingdict["1"][0][1][3],headingdict["1"][0][1][4])

KMCI -94.72505188
32.63339996 -95.64760208


In [17]:
segs2 = []

for heading in range(len(headingdict)):
    for i in headingdict[str(heading+1)]:
        code = i[0][0]
        x1 = i[0][4]
        y1 = i[0][3]
        code2 = i[1][0]
        x2 = i[1][4]
        y2 = i[1][3]

        segs2.append([code,x1,y1,code2,x2,y2])

In [18]:
print(segs2[1])

['K1M5', -86.4769516, 36.59285164, 'KGZH', -87.0469017, 31.4147501]


In [19]:
fields = ['airport1','x1', 'y1','airport2', 'x2', 'y2'] 

with open('CSV/intersect.csv', 'w') as f:
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(segs2)

In [20]:
with open('CSV/intersect-reviewed - intersect.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data[50])

['KGEY', '-108.0880013', '44.5142498', 'OG21', '-120.8230019', '44.19515038', '', '']


In [21]:
df = pd.read_csv('CSV/airport-code.csv')
df.head()

,ident,locale
0,KLAX,"Los Angeles, California"
1,KORD,"Chicago, Illinois"
2,KJFK,"New York, New York"
3,KATL,"Atlanta, Georgia"
4,KSFO,"San Francisco, California"


In [22]:
for entry in range(1,len(data)):

    airport1 = data[entry][0]
    airport2 = data[entry][3]

    search1 = df[df['ident'] == airport1]
    search2 = df[df['ident'] == airport2]

    city1 = search1['locale'].item()
    city2 = search2['locale'].item()

    data[entry][6] = city1
    data[entry][7] = city2

print(data[1])

['KMCI', '-94.72505188', '39.3077507', 'US-6340', '-95.64760208', '32.63339996', 'Kansas City, Missouri', 'Grand Saline, Texas']


In [23]:
fields = data[0]
    
rows = data[1:]
  
with open('CSV/intersectingcitys.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

In [29]:
df = pd.read_csv("/Users/benperry/Documents/Python/Airports/citytocityqq.csv")
df.head()

,airport1,x1,y1,airport2,x2,y2,city1,city2,city to city
0,KMCI,-94.725052,39.307751,US-6340,-95.647602,32.633400,"Kansas City, Missouri","Grand Saline, Texas","Kansas City, Missouri to Grand Saline, Texas"
1,K1M5,-86.476952,36.592852,KGZH,-87.046902,31.414750,"Portland, Tennessee","Evergreen, Alabama","Portland, Tennessee to Evergreen, Alabama"
2,MM28,-98.943600,19.874001,KEOS,-94.391701,36.810801,"Tizayuca, Mexico","Neosho, Missouri","Tizayuca, Mexico to Neosho, Missouri"
3,KVWU,-94.516350,48.150949,KCAO,-103.152500,36.446251,"Waskish, Minnesota","Clayton, New Mexico","Waskish, Minnesota to Clayton, New Mexico"
4,KMSN,-89.335247,43.141851,KCWI,-90.328300,41.832500,"Madison, Wisconsin","Clinton, Iowa","Madison, Wisconsin to Clinton, Iowa"


In [ ]:
fig = go.Figure()

source_to_dest = zip(
    df["y1"],
    df["y2"],
    df["x1"],
    df["x2"],
)

## Loop thorugh each flight entry to add line between source and destination
for slat, dlat, slon, dlon in source_to_dest:
    fig.add_trace(
        go.Scattergeo(
            lat=[slat, dlat],
            lon=[slon, dlon],
            mode="lines",
            line=dict(width=1, color="red"),
            # hovertext="none"
        )
    )

cities = df["airport1"].values.tolist()+df["airport2"].values.tolist()
countries = df["city to city"].values.tolist()
scatter_hover_data = [country + " : "+ city for city, country in zip(cities, countries)]

## Loop thorugh each flight entry to plot source and destination as points.
fig.add_trace(
    go.Scattergeo(
        lon=df["x1"].values.tolist() + df["x2"].values.tolist(),
        lat=df["y1"].values.tolist() + df["y2"].values.tolist(),
        hoverinfo="text",
        text=scatter_hover_data,
        mode="markers",
        marker=dict(size=5, color="blue", opacity=0.1),
    )
)

## Update graph layout to improve graph styling.
fig.update_layout(
    title_text="Airport to Airport",
    margin={"t": 0, "b": 0, "l": 0, "r": 0, "pad": 0},
    showlegend=False,
    geo=dict(
        showland=True,
        landcolor="white",
        countrycolor="grey",
        bgcolor="lightgrey",
        projection_type="orthographic",
        showcountries=True,
        projection_rotation={"lon":-90,"lat":15},
    ),
)

fig.show()